1.分析不归一化的影响

用梯度下降训练

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = torch.tensor([[2, 1000], [3, 2000], [2, 500], [1, 800], [4, 3000]], dtype=torch.float, device=device)
labels = torch.tensor([[19], [31], [14], [15], [43]], dtype=torch.float, device=device)


w = torch.ones(2, 1, requires_grad=True, device=device)
b = torch.ones(1, requires_grad=True, device=device)

epoch = 200
lr = 0.001

for i in range(epoch):
    outputs = inputs @ w + b
    loss = torch.mean(torch.square(outputs - labels))
    print("loss", loss.item())
    loss.backward()
    print("w.grad", w.grad.tolist())
    with torch.no_grad():
        w -= w.grad * lr
        b -= b.grad * lr

    w.grad.zero_()
    b.grad.zero_()

loss 2898583.75
w.grad [[8600.0], [5876040.0]]
loss 102789900402688.0
w.grad [[-51230600.0], [-34991902720.0]]
loss 3.645155836023454e+21
w.grad [[305078960128.0], [208377284132864.0]]
loss 1.2926522439879521e+29
w.grad [[-1816749691371520.0], [-1.240889719093461e+18]]
loss 4.584028345259264e+36
w.grad [[1.0818770005827518e+19], [7.389516401688671e+21]]
loss inf
w.grad [[-6.4425929217374715e+22], [-4.400467570603201e+25]]
loss inf
w.grad [[3.836573017063949e+26], [2.620484796010668e+29]]
loss inf
w.grad [[-2.284685135652703e+30], [-1.5605024929392312e+33]]
loss inf
w.grad [[1.3605332413758115e+34], [9.29281442581589e+36]]
loss inf
w.grad [[-8.101994607286773e+37], [-inf]]
loss inf
w.grad [[inf], [inf]]
loss nan
w.grad [[nan], [nan]]
loss nan
w.grad [[nan], [nan]]
loss nan
w.grad [[nan], [nan]]
loss nan
w.grad [[nan], [nan]]
loss nan
w.grad [[nan], [nan]]
loss nan
w.grad [[nan], [nan]]
loss nan
w.grad [[nan], [nan]]
loss nan
w.grad [[nan], [nan]]
loss nan
w.grad [[nan], [nan]]
loss nan


可以发现对于lights权重 \( w_0 \) 的梯度值为8600，对于distance权重 \( w_1 \) 的梯度值为5876040。\( w_1 \) 的梯度大概是 \( w_0 \) 梯度的1000倍。这是因为 \( w_1 \) 这个权重是作用在distance上的，\( w_0 \) 这个权重是作用在lights上的。distance的值大概是lights值的1000倍。作用到最终loss函数上，进行同样的改变，\( w_1 \) 的变化对loss值的影响就是 \( w_0 \) 的1000倍。所以造成了两个权重的梯度值相差了1000倍。

初始化时，\( w_0 \) 和 \( w_1 \) 都是1，最终我们希望 \( w_0 \) 能调整为2，\( w_1 \) 能调整到0.01。我们看到 \( w_1 \) 的梯度值非常大，如果学习率稍微大一些，1减去学习率乘以梯度值，就跨过了需要调整到的0.01。这就是为什么学习率需要设置的很小。

为了迁就对 \( w_1 \) 的调整，学习率必须设置的很小，但是这会导致对 \( w_0 \) 每次的调整太少，导致训练非常慢。这就是为什么当 loss 降到7左右就很难再下降了。

根本原因就是对 \( w_0 \) 和 \( w_1 \) 的训练，它们共用学习率，但是因为它们对应feature的取值范围不同，导致他们对 loss 函数的影响不同，进而导致它们对loss的梯度的取值范围不同。

2.对feature进行归一化

如果我们让所有feature的取值范围相同，这样所有训练参数对loss函数的影响就相同了，计算得到的梯度都差不多，就可以用统一的学习率来进行调整了。 对于bias而言，它的系数为1，相当于它的输入feature大小永远都是1。那么我们就把其他feature都调整到1左右。最简单的做法，就是让输入feature都除以这个feature的最大值，这样所有feature的取值都是0到1之间了。 我们试一下这样是否可以改进训练的稳定性：

In [5]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = torch.tensor([[2, 1000], [3, 2000], [2, 500], [1, 800], [4, 3000]], dtype=torch.float, device=device)
labels = torch.tensor([[19], [31], [14], [15], [43]], dtype=torch.float, device=device)

#进行归一化
inputs = inputs / torch.tensor([4, 3000], device=device)
print(inputs )

w = torch.ones(2, 1, requires_grad=True, device=device)
b = torch.ones(1, requires_grad=True, device=device)

epoch = 1000
lr = 0.5

for i in range(epoch):
    outputs = inputs @ w + b
    loss = torch.mean(torch.square(outputs - labels))
    print("loss", loss.item())
    loss.backward()
    print("w.grad", w.grad.tolist())
    with torch.no_grad():
        w -= w.grad * lr
        b -= b.grad * lr

    w.grad.zero_()
    b.grad.zero_()

tensor([[0.5000, 0.3333],
        [0.7500, 0.6667],
        [0.5000, 0.1667],
        [0.2500, 0.2667],
        [1.0000, 1.0000]], device='cuda:0')
loss 609.12255859375
w.grad [[-31.823333740234375], [-28.171558380126953]]
loss 276.1471252441406
w.grad [[18.71324920654297], [14.430889129638672]]
loss 130.74789428710938
w.grad [[-14.165597915649414], [-13.107969284057617]]
loss 66.27214813232422
w.grad [[7.567932605743408], [5.258292198181152]]
loss 36.886844635009766
w.grad [[-6.48609733581543], [-6.472208023071289]]
loss 22.86341667175293
w.grad [[2.8819427490234375], [1.481224775314331]]
loss 15.68339729309082
w.grad [[-3.1059412956237793], [-3.48284912109375]]
loss 11.647061347961426
w.grad [[0.9495362043380737], [-0.00957345962524414]]
loss 9.129935264587402
w.grad [[-1.585668921470642], [-2.0832316875457764]]
loss 7.404508113861084
w.grad [[0.18418478965759277], [-0.5427906513214111]]
loss 6.133531093597412
w.grad [[-0.8760247826576233], [-1.3865255117416382]]
loss 5.1517028808593

3.对特征进行标准化

实际上在深度学习里，更常用的是对特征进行标准化处理。也就是对每个feature减去自己的均值，再除以自己的标准差。这样就把这个feature转化为均值为0，标准差为1的分布了。在归一化操作里，是对每个feature除以这个feature所有样本中绝对值最大的值。只有这一个值决定缩放大小。但这个值有可能是个异常值。相比之下标准化处理会考虑所有样本的分布情况，避免缩放受异常值的影响，训练起来会更稳定。

In [7]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = torch.tensor([[2, 1000], [3, 2000], [2, 500], [1, 800], [4, 3000]], dtype=torch.float, device=device)
labels = torch.tensor([[19], [31], [14], [15], [43]], dtype=torch.float, device=device)

#计算特征的均值和标准差
mean = inputs.mean(dim=0)
std = inputs.std(dim=0)
print("mean", mean)
print("std", std)
#对特征进行标准化
inputs_norm = (inputs-mean)/std

w = torch.ones(2, 1, requires_grad=True, device=device)
b = torch.ones(1, requires_grad=True, device=device)

epoch = 1000
lr = 0.5

for i in range(epoch):
    outputs = inputs_norm @ w + b
    loss = torch.mean(torch.square(outputs - labels))
    print("loss", loss.item())
    loss.backward()
    print("w.grad", w.grad.tolist())
    with torch.no_grad():
        w -= w.grad * lr
        b -= b.grad * lr

    w.grad.zero_()
    b.grad.zero_()

mean tensor([   2.4000, 1460.0000], device='cuda:0')
std tensor([   1.1402, 1028.5913], device='cuda:0')
loss 635.2097778320312
w.grad [[-15.60400390625], [-16.726499557495117]]
loss 25.922632217407227
w.grad [[9.087748527526855], [8.043952941894531]]
loss 8.4130220413208
w.grad [[-4.0536723136901855], [-5.024291038513184]]
loss 3.343095541000366
w.grad [[2.8564584255218506], [1.9538871049880981]]
loss 1.7868238687515259
w.grad [[-0.8548362255096436], [-1.6941308975219727]]
loss 1.2350934743881226
w.grad [[1.0655667781829834], [0.2851123511791229]]
loss 0.980950653553009
w.grad [[0.005011647939682007], [-0.7207277417182922]]
loss 0.8237902522087097
w.grad [[0.5270583629608154], [-0.14780157804489136]]
loss 0.7054709792137146
w.grad [[0.21328718960285187], [-0.4142606258392334]]
loss 0.6080939769744873
w.grad [[0.3450268507003784], [-0.2385251522064209]]
loss 0.5252779126167297
w.grad [[0.24310298264026642], [-0.299537718296051]]
loss 0.4540571868419647
w.grad [[0.26724934577941895], [-

4.预测时的归一化

有一点要特别注意，假如你在训练时对数据做了归一化，那么你一定要记录你做归一化时的参数。在对数据进行预测时，首先需要先对feature用同样的参数进行归一化，然后再带入模型，得到预测值。

In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = torch.tensor([[2, 1000], [3, 2000], [2, 500], [1, 800], [4, 3000]], dtype=torch.float, device=device)
labels = torch.tensor([[19], [31], [14], [15], [43]], dtype=torch.float, device=device)

#计算每个特征的均值和标准差
mean = inputs.mean(dim=0)
std = inputs.std(dim=0)
#对特征进行标准化
inputs = (inputs-mean)/std

w = torch.ones(2, 1, requires_grad=True, device=device)
b = torch.ones(1, requires_grad=True, device=device)

epoch = 2000
lr = 0.1

for i in range(epoch):
    outputs = inputs @ w + b
    loss = torch.mean(torch.square(outputs - labels))
    print("loss", loss.item())
    loss.backward()
    print("w.grad", w.grad.tolist())
    with torch.no_grad():
        w -= w.grad * lr
        b -= b.grad * lr

    w.grad.zero_()
    b.grad.zero_()

# 对新采集的数据进行预测
new_input = torch.tensor([[3,2500]],dtype=torch.float,device=device)
# 对于新的数据进行预测时，同样要进行标准化
new_input = (new_input-mean)/std
# 预测
predict = new_input @ w + b
# 打印预测结果
print("Predict:",predict.tolist()[0][0])

loss 635.2097778320312
w.grad [[-15.60400390625], [-16.726499557495117]]
loss 393.7582092285156
w.grad [[-10.665653228759766], [-11.772409439086914]]
loss 246.2174530029297
w.grad [[-7.240628719329834], [-8.331866264343262]]
loss 155.147216796875
w.grad [[-4.865854740142822], [-5.941790580749512]]
loss 98.46870422363281
w.grad [[-3.219943046569824], [-4.280791282653809]]
loss 62.9586296081543
w.grad [[-2.0798492431640625], [-3.1258230209350586]]
loss 40.59088897705078
w.grad [[-1.290771484375], [-2.3220789432525635]]
loss 26.439193725585938
w.grad [[-0.7452720403671265], [-1.7621188163757324]]
loss 17.45210838317871
w.grad [[-0.36879947781562805], [-1.3713887929916382]]
loss 11.725476264953613
w.grad [[-0.10959792137145996], [-1.0981289148330688]]
loss 8.064153671264648
w.grad [[0.06824113428592682], [-0.9064292311668396]]
loss 5.714572429656982
w.grad [[0.18964245915412903], [-0.7713615894317627]]
loss 4.199885368347168
w.grad [[0.27190011739730835], [-0.6756294965744019]]
loss 3.2175

5.为什么归一化不会影响模型

你可能好奇，归一化实际改变了数据，为什么不影响训练模型呢？这是因为归一化仅对参数空间进行了可逆的线性变换，模型的理论表达能力不变，不改变数据的本质关系。这一现象类似于“换单位不会影响物理规律”。 归一化不会影响深度学习模型的训练结果，因为它只是数据的线性变换，保留了所有必要的信息，模型可以通过权重调整完全补偿这种变换。

6.归一化还是标准化

严格来说，归一化指的是将数据变化调整到[-1,1]或者[0,1]之间。标准化是将数据减去均值除以标准差。但是在机器学习里，由于历史原因，都是用Normalization来表示。如果有人说他对数据进行了归一化，但是实际代码里是做了标准化，你也不用感到奇怪。

7.什么时候用归一化

因为对feature进行归一化会让训练更稳定，且不会带来任何坏处。基本上所有的深度学习的模型都默认会对feature进行归一化操作。